In [35]:
import cv2 as cv
import sys
import numpy as np

cap = cv.VideoCapture(0)

video_file = './chess.mp4'

if not cap.isOpened():
    print("Camera Open Failed")
    sys.exit()

width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv.CAP_PROP_FPS)

four_cc = cv.VideoWriter_fourcc(*'DIVX')
wait_msec = int(1000 / fps)

out = cv.VideoWriter(video_file, four_cc, fps, (width, height))
recording = False

if not out.isOpened():
    print('File Open Failed')
    cap.release()
    sys.exit()

while True:
    valid, frame = cap.read()
    key = cv.waitKey(wait_msec)

    if not valid:
        print("Error occured")
        break

    if recording:
        out.write(frame)
        cv.circle(frame, (100, 15), radius=10, color=(0, 0, 255), thickness=-1)
        cv.putText(frame, 'Recording', (10, 20), cv.FONT_HERSHEY_DUPLEX, 0.5, color=(0, 0, 255))
    else:
        cv.putText(frame, 'Press Space to start recording', (80, 50), cv.FONT_HERSHEY_TRIPLEX, 0.5, color=(0, 0, 0), thickness = 2)
    
    cv.imshow('Calibration', frame)

    if key == ord(' '):
        if not recording:
            recording = True
            continue
        else:
            recording = False
            continue

    elif key == 27:
        break

cap.release()
out.release()
cv.destroyAllWindows()


In [36]:
def select_img_from_video(video_file, board_pattern, select_all=False, wait_msec=50, wnd_name='Camera Calibration'):
    # Open a video
    video = cv.VideoCapture(video_file)
    assert video.isOpened()

    # Select images
    img_select = []
    while True:
        # Grab an images from the video
        valid, img = video.read()
        if not valid:
            break

        if select_all:
            img_select.append(img)
        else:
            # Show the image
            display = img.copy()
            cv.putText(display, f'NSelect: {len(img_select)}', (10, 25), cv.FONT_HERSHEY_DUPLEX, 0.6, (0, 255, 0))
            cv.imshow(wnd_name, display)

            # Process the key event
            key = cv.waitKey(wait_msec)
            if key == ord(' '):             # Space: Pause and show corners
                complete, pts = cv.findChessboardCorners(img, board_pattern)
                cv.drawChessboardCorners(display, board_pattern, pts, complete)
                cv.imshow(wnd_name, display)
                key = cv.waitKey()
                if key == ord('\r'):
                    img_select.append(img) # Enter: Select the image
            if key == 27:                  # ESC: Exit (Complete image selection)
                break

    cv.destroyAllWindows()
    return img_select

def calib_camera_from_chessboard(images, board_pattern, board_cellsize, K=None, dist_coeff=None, calib_flags=None):
    # Find 2D corner points from given images
    img_points = []
    for img in images:
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        complete, pts = cv.findChessboardCorners(gray, board_pattern)
        if complete:
            img_points.append(pts)
    assert len(img_points) > 0

    # Prepare 3D points of the chess board
    obj_pts = [[c, r, 0] for r in range(board_pattern[1]) for c in range(board_pattern[0])]
    obj_points = [np.array(obj_pts, dtype=np.float32) * board_cellsize] * len(img_points) # Must be `np.float32`

    # Calibrate the camera
    return cv.calibrateCamera(obj_points, img_points, gray.shape[::-1], K, dist_coeff, flags=calib_flags)

if __name__ == '__main__':
    video_file = './chess.mp4'
    board_pattern = (10, 7)
    board_cellsize = 0.025

    img_select = select_img_from_video(video_file, board_pattern)
    assert len(img_select) > 0, 'There is no selected images!'
    rms, K, dist_coeff, rvecs, tvecs = calib_camera_from_chessboard(img_select, board_pattern, board_cellsize)

    # Print calibration results
    print('## Camera Calibration Results')
    print(f'* The number of selected images = {len(img_select)}')
    print(f'* RMS error = {rms}')
    print(f'* Camera matrix (K) = \n{K}')
    print(f'* Distortion coefficient (k1, k2, p1, p2, k3, ...) = {dist_coeff.flatten()}')


## Camera Calibration Results
* The number of selected images = 28
* RMS error = 1.960827951378868
* Camera matrix (K) = 
[[593.50380396   0.         311.06884721]
 [  0.         591.81946408 241.45158454]
 [  0.           0.           1.        ]]
* Distortion coefficient (k1, k2, p1, p2, k3, ...) = [-0.03603741  0.28862958  0.00094713 -0.01065978 -0.57473369]


In [28]:
print(K)
print(dist_coeff)

[[748.4333166    0.         362.45631548]
 [  0.         812.24517367 370.66192862]
 [  0.           0.           1.        ]]
[[-0.16926594  2.4668213   0.09580364  0.00873149 -5.31619604]]


In [37]:
video_file = './chess.mp4'
# K = np.array([[488., 0, 476.0614994349778],
#               [0, 431.2395555913084, 288.7602152621297],
#               [0, 0, 1]]) # Derived from `calibrate_camera.py`
# dist_coeff = np.array([-0.2852754904152874, 0.1016466459919075, -0.0004420196146339175, 0.0001149909868437517, -0.01803978785585194])

# Open a video
video = cv.VideoCapture(video_file)
assert video.isOpened(), 'Cannot read the given input, ' + video_file

# Run distortion correction
show_rectify = True
map1, map2 = None, None
while True:
    # Read an image from the video
    valid, img = video.read()
    if not valid:
        break

    # Rectify geometric distortion (Alternative: `cv.undistort()`)
    info = "Original"
    if show_rectify:
        if map1 is None or map2 is None:
            map1, map2 = cv.initUndistortRectifyMap(K, dist_coeff, None, None, (img.shape[1], img.shape[0]), cv.CV_32FC1)
        img = cv.remap(img, map1, map2, interpolation=cv.INTER_LINEAR)
        info = "Rectified"
    cv.putText(img, info, (10, 25), cv.FONT_HERSHEY_DUPLEX, 0.6, (0, 255, 0))

    # Show the image and process the key event
    cv.imshow("Geometric Distortion Correction", img)
    key = cv.waitKey(50)
    if key == ord(' '):     # Space: Pause
        key = cv.waitKey()
    if key == 27:           # ESC: Exit
        break
    elif key == ord('\t'):  # Tab: Toggle the mode
        show_rectify = not show_rectify

video.release()
cv.destroyAllWindows()
